In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score

In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_training_path = "data/dataset_v2/training"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_training_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_training_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_train = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_train)}")

100%|██████████| 8720/8720 [00:25<00:00, 342.70it/s]


8720
8720
8720
8720

number of rows: 8720


In [5]:
df_recommender_train = df_recommender_train[(df_recommender_train["recommend_indexes"] != -1) & (df_recommender_train["turns"].apply(lambda x: len(x) > 0))]

In [6]:
apps_training_path = "data/master_app_data_V1_true.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())

In [7]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_train['recommended_app_name'] = df_recommender_train.apply(fix_recommended_apps_names, axis=1)

In [8]:
max_existing_length = max(len(item) for item in all_apps)  # Max length in current array
new_dtype = f'<U{max_existing_length}'

def candidate_creator(row):
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_app_name"]]), 24, replace=False).astype(new_dtype)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_train['candidate_apps'] = df_recommender_train.apply(candidate_creator, axis=1)

In [9]:
df_recommender_train["recommended_app_name"][874] 

'audiomack-stream music offline'

In [10]:
df_recommender_train["turns"][874][df_recommender_train["recommend_indexes"][874]]

'computer: in that case, i recommend trying audiomack-stream music offline app.'

In [11]:
df_recommender_train["candidate_apps"][874]

array(['strawberry shortcake sweet shop', 'jetpack joyride',
       "five nights at freddy's 4", 'hungry dragon', 'vlc for android',
       'little big city 2', 'real driving school', 'dawn break -origin-',
       'words of wonders: crossword', 'asphalt nitro',
       'one booster - antivirus, booster, phone cleaner',
       'spongebob: sponge on the run', 'pixellab - text on pictures',
       'mini world: creata', 'audiomack-stream music offline',
       'bilibili comics - read comics', 'toca life: neighborhood',
       'car mechanic simulator 21', 'pluto tv - live tv and movies',
       'world at arms', "alto's adventure", 'move people',
       'lego® ninjago: shadow of ronin', 'asphalt 8 - car racing game',
       'playstation app'], dtype='<U50')

In [12]:
conversation_validation_path = "data/dataset_v2/validation"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_validation_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_validation_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_validation = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_validation)}")

100%|██████████| 1285/1285 [00:04<00:00, 300.61it/s]

1285
1285
1285
1285

number of rows: 1285


In [13]:
df_recommender_validation = df_recommender_validation[(df_recommender_validation["recommend_indexes"] != -1) & (df_recommender_validation["turns"].apply(lambda x: len(x) > 0))]

In [14]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_validation['recommended_app_name'] = df_recommender_validation.apply(fix_recommended_apps_names, axis=1)

In [15]:
max_existing_length = max(len(item) for item in all_apps)  # Max length in current array
new_dtype = f'<U{max_existing_length}'

def candidate_creator(row):
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_app_name"]]), 24, replace=False).astype(new_dtype)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_validation['candidate_apps'] = df_recommender_validation.apply(candidate_creator, axis=1)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:", "candidate_apps:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

In [17]:
prompt_validation = []
recommend_validation = []
for _, row in df_recommender_validation.iterrows():
    prompt = ""
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_validation.append(prompt)
            recommend_validation.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

In [18]:
prompt_train = []
recommend_train = []
for _, row in df_recommender_train.iterrows():
    prompt = ""
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_train.append(prompt)
            recommend_train.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")
            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

In [19]:
recommend_encodings["input_ids"][10]

tensor([   36, 21391, 13634,     7,  1120,     5,     1,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])

In [20]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [21]:
training_args = TrainingArguments(
    output_dir="models/T5_candidate_apps",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=1000,#self.cfg.save_steps,
    eval_steps=1000, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
1000,0.147200,0.090282
2000,0.081200,0.067263
3000,0.063200,0.063722


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [23]:
torch.cuda.empty_cache()

In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_test_path = "data/dataset_v2/testing"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_test_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_test_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_test = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_test)}")

  0%|          | 0/2557 [00:00<?, ?it/s]

100%|██████████| 2557/2557 [00:07<00:00, 352.88it/s]

2557
2557
2557
2557

number of rows: 2557


In [5]:
df_recommender_test = df_recommender_test[(df_recommender_test["recommend_indexes"] != -1) & (df_recommender_test["turns"].apply(lambda x: len(x) > 0))]

In [6]:
apps_training_path = "data/master_app_data_V1_true.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())

In [7]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_test['recommended_app_name'] = df_recommender_test.apply(fix_recommended_apps_names, axis=1)

In [8]:
import pandas as pd
from fuzzywuzzy import fuzz

# Ensure that you've already defined 'apps_training_path' to point to your data file
cols = ['app_name', 'app_type', 'app_category']
df_app = pd.read_csv(apps_training_path, usecols=cols)
df_apps = df_app.applymap(lambda x: x.lower() if isinstance(x, str) else x)

def filter_candidate_apps(rec_app_name):
    candidate_apps = set()  # Using a set to avoid duplicates
    df_rec_app = df_apps[df_apps['app_name'] == rec_app_name.lower()]
    
    if df_rec_app.empty:
        print("No matching app found.")
        return []
    
    recommended_app_type = df_rec_app['app_type'].iloc[0]
    recommended_app_category = df_rec_app['app_category'].iloc[0]
    
    df_same_category = df_apps[df_apps['app_category'] == recommended_app_category]
    df_different_category = df_apps[df_apps['app_category'] != recommended_app_category]

    # same category, same type
    for _, row in df_same_category.iterrows():
        if fuzz.ratio(row["app_type"], recommended_app_type) > 70:
            candidate_apps.add(row['app_name'])

    # only same category
    if len(candidate_apps) < 25:
        for _, row in df_same_category.iterrows():
            if len(candidate_apps) >= 25:
                break
            candidate_apps.add(row['app_name'])

    # different category, same type
    if len(candidate_apps) < 25:
        for _, row in df_different_category.iterrows():
            if len(candidate_apps) >= 25:
                break
            if fuzz.ratio(row["app_type"], recommended_app_type) > 70:
                candidate_apps.add(row['app_name'])

    # all
    if len(candidate_apps) < 25:
        for _, row in df_apps.iterrows():
            if len(candidate_apps) >= 25:
                break
            candidate_apps.add(row['app_name'])

    return list(candidate_apps)  # Converting back to list if needed for downstream processes

/tmp/ipykernel_488036/766425766.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_apps = df_app.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [16]:
max_existing_length = max(len(item) for item in all_apps)  # Max length in current array
new_dtype = f'<U{max_existing_length}'

def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_app_name"]]), 24, replace=False).astype(new_dtype) # filter_candidate_apps(row["recommended_app_name"])
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_test['candidate_apps'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [17]:
prompt_test = []
recommend_test = []
candidate_apps = []
true_candidate_index = []
for _, row in df_recommender_test.iterrows():
    # creating candidate apps
    candidates = []
    for index, candidate_app in enumerate(row["candidate_apps"].tolist()):
        candidates.append(candidate_app + " app.")
        if candidate_app == row["recommended_app_name"]:
            true_candidate_index.append(index)
    candidate_apps.append(candidates)
    prompt = ""
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "models/T5_candidate_apps")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:",  "candidate_apps:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

In [19]:
# bleu = evaluate.load("bleu")

In [20]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" app.", "")  for generation in generations_predicted]
    generation_batch = [generation.replace(" app.", "") for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [21]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=4, threshold=95)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

100%|██████████| 634/634 [06:38<00:00,  1.59it/s]

success_rate:  0.860410094637224


In [48]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [49]:
scores = recommender_rank(prompt_test, candidate_apps, model, tokenizer, batch_size=8)

100%|██████████| 7925/7925 [30:41<00:00,  4.30it/s]


In [50]:
[top_k_accuracy_score(true_candidate_index, scores, k=k) for k in range(1, 11)]

[0.5670347003154574,
 0.7022870662460567,
 0.7567034700315457,
 0.8000788643533123,
 0.8324132492113565,
 0.8485804416403786,
 0.862776025236593,
 0.874211356466877,
 0.8848580441640379,
 0.8955047318611987]

In [51]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_apps[0]))] for index in true_candidate_index]

In [52]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[0.5670347003154574,
 0.6523694422220071,
 0.6795776441147515,
 0.6982584096463261,
 0.7107670571988626,
 0.7165259273374387,
 0.7212577885361768,
 0.7248652338937424,
 0.7280702062450545,
 0.731147789175095]

: 